DCGAN相比于GAN做出的改变是：

1、G网络中使用转置卷积（transposed convolutional layer）进行上采样，D网络中用加入stride的卷积代替pooling。

2、在D和G中均使用batch normalization

3、去掉FC层，使网络变为全卷积网络

4、G网络中使用ReLU作为激活函数，最后一层使用tanh

5、D网络中使用LeakyReLU作为激活函数

具体的结构图下所示：

Generator的结构：

![DCGAN中的G结构图](../images/DCGAN/DCGAN_G.png)

Discriminator的结构：

![DCGAN中的D结构图](../images/DCGAN/DCGAN_D.png)


然后几个心得是：

1、GAN真的难训练，learning_rate尽量小点

2、自己尝试过的将最后的Conv层换成Linear层，发现效果真的差...

3、在没看WGAN之前，以DCGAN来说的话，似乎没有什么指标可以让我们定量的分析训练过程中GAN表现的好坏。

4、看loss很容易看到傻眼... 有时候loss真的会小到让你怀疑网络是不是出错了...

5、最后也是最重要的一点就是：写完了代码，并且顺利运行了，但是结果不好，先别急着调参，因为很有可能是你的代码写的有问题。

数据：https://pan.baidu.com/s/1eSifHcA  提取码：g5qa 

以上内容参考 https://zhuanlan.zhihu.com/p/24767059 

## 结果展示


训练10次的结果

![DCGAN10次迭代图](./DCGAN_result/fake_images-10.png)

训练50次的结果

![DCGAN50次迭代图](./DCGAN_result/fake_images-50.png)

训练100次的结果

![DCGAN100次迭代图](./DCGAN_result/fake_images-100.png)

训练200次的结果

![DCGAN200次迭代图](./DCGAN_result/fake_images-200.png)

训练300次的结果

![DCGAN300次迭代图](./DCGAN_result/fake_images-300.png)

训练400次的结果

![DCGAN400次迭代图](./DCGAN_result/fake_images-400.png)

训练500次的结果

![DCGAN500次迭代图](./DCGAN_result/fake_images-500.png)

In [1]:
import torch
import torch.nn as nn
from torch.autograd import Variable
from torchvision import transforms
from torch.utils import data
import os
from PIL import Image
from torch import optim
import torch.nn.functional as F
from torchvision.utils import save_image

In [2]:
# 构建生成模型
class Generator(nn.Module):
    def __init__(self, noise_dim=100, conv_dim=64, g_img_size=64):
        super(Generator, self).__init__()
        self.fc = nn.ConvTranspose2d(noise_dim, conv_dim*16, kernel_size=int(g_img_size/16), stride=1, padding=0)
        self.deconv2 = nn.ConvTranspose2d(conv_dim*16, conv_dim*8, kernel_size=4, stride=2, padding=1)
        self.bn2 = nn.BatchNorm2d(conv_dim * 8)
        self.deconv3 = nn.ConvTranspose2d(conv_dim*8, conv_dim*4, kernel_size=4, stride=2, padding=1)
        self.bn3 = nn.BatchNorm2d(conv_dim * 4)
        self.deconv4 = nn.ConvTranspose2d(conv_dim*4, conv_dim*2, kernel_size=4, stride=2, padding=1)
        self.bn4 = nn.BatchNorm2d(conv_dim * 2)
        self.deconv5 = nn.ConvTranspose2d(conv_dim*2, 3, kernel_size=4, stride=2, padding=1)


    def forward(self, x):
        x = x.view(x.size(0), x.size(1), 1, 1)
        A1 = self.fc(x)
        A2 = F.relu(self.bn2(self.deconv2(A1)))
        A3 = F.relu(self.bn3(self.deconv3(A2)))
        A4 = F.relu(self.bn4(self.deconv4(A3)))
        y_hat = F.tanh(self.deconv5(A4))
        return y_hat

In [3]:
# 构建判别模型
class Discriminator(nn.Module):
    def __init__(self, input_chanel=3, conv_dim=64, d_img_size=64):
        super(Discriminator, self).__init__()
        self.conv1 = nn.Conv2d(input_chanel, conv_dim * 2, kernel_size=4, stride=2, padding=1)
        self.conv2 = nn.Conv2d(conv_dim * 2, conv_dim * 4, kernel_size=4, stride=2, padding=1)
        self.bn2 = nn.BatchNorm2d(conv_dim * 4)
        self.conv3 = nn.Conv2d(conv_dim * 4, conv_dim * 8, kernel_size=4, stride=2, padding=1)
        self.bn3 = nn.BatchNorm2d(conv_dim * 8)
        self.conv4 = nn.Conv2d(conv_dim * 8, conv_dim * 16, kernel_size=4, stride=2, padding=1)
        self.bn4 = nn.BatchNorm2d(conv_dim * 16)
        self.fc = nn.Conv2d(conv_dim * 16, 1, int(d_img_size / 16), 1, 0)

    def forward(self, x):
        A1 = F.leaky_relu(self.conv1(x))
        A2 = F.leaky_relu(self.bn2(self.conv2(A1)), 0.05)
        A3 = F.leaky_relu(self.bn3(self.conv3(A2)), 0.05)
        A4 = F.leaky_relu(self.bn4(self.conv4(A3)), 0.05)
        y_hat = F.sigmoid(self.fc(A4).squeeze())
        return y_hat

In [4]:
# 数据集处理
class ImageDataset(data.Dataset):
    def __init__(self, path, transform=None):
        """
            path 是存在图像的文件夹。
        """
        self.images = list(map(lambda x: os.path.join(path, x), os.listdir(path)))
        self.transform = transform

    def __getitem__(self, index):
        image_file = self.images[index]
        image = Image.open(image_file).convert('RGB')
        if self.transform is not None:
            image = self.transform(image)
        return image

    def __len__(self):
        return len(self.images)


def get_dataset(path, img_scale, batch_size):
    transform = transforms.Compose([
        transforms.Resize(img_scale),
        # 也可以使用 Scale
        transforms.ToTensor(),
        transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
    ])

    dataset = ImageDataset(path, transform)
    data_loader = data.DataLoader(dataset=dataset,
                                  batch_size=batch_size,
                                  shuffle=True,
                                  drop_last=True)
    return data_loader

In [5]:
# 工具函数:

# 生成 噪音 z
def gen_noisy(batch_size, noisy_dim):
    return torch.randn(batch_size, noisy_dim)


# tensor to variable
def to_variable(x):
    x = Variable(x)
    if torch.cuda.is_available():
        x = x.cuda()
    return x

In [6]:
# 这里可以定义一个 Config 类，用来保存这些超参数
class Config(object):
    def __init__(self):
        self.batch_size = 128
        self.image_path = './faces/' # 图像数据存放的文件，如有需要请自行调整。
        self.noisy_dim = 100
        self.G_lr = 2*1e-6
        self.D_lr = 2*1e-6
        self.EPOCH = 500
        self.img_scale = 64
        self.k_step = 5
        self.use_gpu = True
        self.g_img_size = 64
        self.d_img_size = 64

In [7]:
# 训练阶段 在正式开始训练前，我们先来看看写完的 Generator 和 Discriminator
config = Config()

G = Generator(noise_dim=config.noisy_dim, g_img_size=config.g_img_size)
if config.use_gpu:
    G = G.cuda()
    
D = Discriminator(d_img_size=config.d_img_size)
if config.use_gpu:
    D = D.cuda()

In [8]:
print("Generator的结构是：")
print(G)

Generator的结构是：
Generator (
  (fc): ConvTranspose2d(100, 1024, kernel_size=(4, 4), stride=(1, 1))
  (deconv2): ConvTranspose2d(1024, 512, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1))
  (bn2): BatchNorm2d(512, eps=1e-05, momentum=0.1, affine=True)
  (deconv3): ConvTranspose2d(512, 256, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1))
  (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True)
  (deconv4): ConvTranspose2d(256, 128, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1))
  (bn4): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True)
  (deconv5): ConvTranspose2d(128, 3, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1))
)


In [9]:
print("Discriminator的结构是：")
print(D)

Discriminator的结构是：
Discriminator (
  (conv1): Conv2d(3, 128, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1))
  (conv2): Conv2d(128, 256, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1))
  (bn2): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True)
  (conv3): Conv2d(256, 512, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1))
  (bn3): BatchNorm2d(512, eps=1e-05, momentum=0.1, affine=True)
  (conv4): Conv2d(512, 1024, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1))
  (bn4): BatchNorm2d(1024, eps=1e-05, momentum=0.1, affine=True)
  (fc): Conv2d(1024, 1, kernel_size=(4, 4), stride=(1, 1))
)


In [ ]:
# 正式开始训练的阶段：

train_data_loader = get_dataset(config.image_path, config.img_scale, config.batch_size)
loss_fn = torch.nn.BCELoss()
g_optimizer = optim.Adam(G.parameters(), lr=config.G_lr)
d_optimizer = optim.Adam(D.parameters(), lr=config.D_lr)

for epoch in range(config.EPOCH):
    g_total_loss = torch.FloatTensor([0])
    d_total_loss = torch.FloatTensor([0])
    count = 0
    
    for i, data in enumerate(train_data_loader):
        count += 1
        true_inputs = data
        images = to_variable(true_inputs)
        batch_size = images.size(0)

        z = to_variable(gen_noisy(batch_size, config.noisy_dim))

        real_labels = to_variable(torch.ones(batch_size))
        fake_labels = to_variable(torch.zeros(batch_size))


        ###          train D           ###
        outputs = D(images)
        d_loss_real = loss_fn(outputs, real_labels)
        real_score = outputs

        fake_images = G(z)
        outputs = D(fake_images)
        d_loss_fake = loss_fn(outputs, fake_labels)
        fake_score = outputs

        d_loss = d_loss_real + d_loss_fake
        # count total loss
        d_total_loss += d_loss.data[0]
        D.zero_grad()
        d_loss.backward()
        d_optimizer.step()


        ###          train G           ###
        z = to_variable(gen_noisy(batch_size, config.noisy_dim))
        fake_images = G(z)
        outputs = D(fake_images)
        g_loss = loss_fn(outputs, real_labels)
        g_total_loss += g_loss.data[0]
        G.zero_grad()
        g_loss.backward()
        g_optimizer.step()

        if (i + 1) % 150 == 0:
            print('Epoch [%d/%d], Step[%d/%d], d_loss: %.4f, '
                  'g_loss: %.4f, D(x): %.2f, D(G(z)): %.2f'
                  % (epoch+1, config.EPOCH, i + 1, count, d_loss.data[0], g_loss.data[0],
                     real_score.data.mean(), fake_score.data.mean()))
    print('Epoch [%d/%d]'% (epoch, config.EPOCH))
    print('D 的 total loss', d_total_loss / count)
    print('G 的 total loss', g_total_loss / count)
    # Save real images

    fake_images = fake_images.view(fake_images.size(0), 3, 64, 64)
    # 注意 这里需要创建 test_DCGAN 文件夹，否则会报错
    save_image(fake_images.data, './test_DCGAN/fake_images-%d.png' % (epoch + 1))
